<a href="https://colab.research.google.com/github/AvdeevMike/Bald-not_bald/blob/main/HairHealth_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import drive
import os

# Подключаем Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/HairHealth/'

# Определяем пути к папкам с фотографиями
train_dir = '/content/drive/My Drive/HairHealth/train'
test_dir = '/content/drive/My Drive/HairHealth/test'

# Создаем генераторы изображений для обучения и тестирования
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Загружаем обучающие и тестовые изображения с помощью генераторов
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        classes=['nb', 'bald']
)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        classes=['nb', 'bald']
)

# Создаем модель нейронной сети
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Добавляем модель Inception в нашу модель
inception_model = keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
inception_model.trainable = False

# Компилируем модель
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Обучаем модель
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator))

from PIL import Image

# Путь к папке с фотографиями
photos_dir = '/content/drive/My Drive/HairHealth/chek/'

# Получаем список файлов в папке
photo_files = os.listdir(photos_dir)

# Классифицируем каждую фотографию
for photo_file in photo_files:
    # Полный путь к фотографии
    photo_path = os.path.join(photos_dir, photo_file)

    # Загружаем и изменяем размер фотографии
    photo = Image.open(photo_path)
    photo = photo.resize((150, 150))

    # Преобразуем изображение в массив numpy и нормализуем его
    img_array = np.array(photo) / 255.0

    # Добавляем размерность пакета (batch dimension)
    img_array = np.expand_dims(img_array, axis=0)

    # Классифицируем фотографию
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print(photo_file, " - Лысый человек")
    else:
        print(photo_file, " - Не лысый человек")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/10
8/8 [==============================] - 73s 9s/step - loss: 0.6863 - accuracy: 0.5688 - val_loss: 0.6891 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 17s 2s/step - loss: 0.7182 - accuracy: 0.5375 - val_loss: 0.5606 - val_accuracy: 0.7750
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: 0.5971 - accuracy: 0.7063 - val_loss: 0.5690 - val_accuracy: 0.8000
Epoch 4/10
8/8 [==============================] - 18s 2s/step - loss: 0.4958 - accuracy: 0.7250 - val_loss: 0.4452 - val_accuracy: 0.8250
Epoch 5/10
8/8 [==============================] - 18s 2s/step - loss: 0.3896 - accuracy: 0.8313 - val_loss: 0.6699 - val_accuracy: 0.7000
Epoch 6/10
8/8 [==============================] - 18s 2s/step - loss: 0.2685 - accuracy: 0.9125 - va